## Análise dos casos de transmissão de dengue no estado do Rio de Janeiro

Aluno: Rodrigo dos Santos Pinto da Cunha  
Profº: Fernando Masanori  

Trabalho de análise de dados referente às matérias de Estrutura de Dados e Laboratório de Banco de Dados da FATEC de São José dos Campos Profº Jessen Vidal. Este trabalho tem por objetivo fazer uma análise da incidência dos casos de dengue no estado do Rio de Janeiro.
A fonte de dados utilizada foi o portal [Info Dengue](https://info.dengue.mat.br/) o qual disponibiliza uma [API](https://info.dengue.mat.br/services/api/doc) para acesso às bases de dados em diversos formatos como JSON, CSV, Excel, etc.. As [informações](https://info.dengue.mat.br/informacoes/) deste portal são provenientes de diferentes fontes como clima, redes sociais e dados oficiais de notificação. Para este trabalho foi criado um [shell script](#script) que consolida a informação de todos os munícipios de um estado em um único arquivo permitindo a análise da informação pelo Pandas.

*OBS: O framework [Plotly](https://plot.ly/python/offline/), utilizado para plotar os gráficos neste notebook, apresentou um problema no qual ele parou de exibir os gráficos. Para contornar este problema é preciso iniciar o notebook com a linha de comando `jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10`. Conforme solução sugerida [neste link](https://stackoverflow.com/a/48709863).*
***

Instalando algumas bibliotecas necessárias para a análise.

In [ ]:
!pip install PeakUtils

Bibliotecas e configurações iniciais

In [4]:
import locale
import pandas as pd
import datetime as dt
import calendar
import plotly.offline as py
import plotly.graph_objs as go
import peakutils
import folium
from branca.colormap import LinearColormap
py.init_notebook_mode(connected=True)
locale.setlocale(locale.LC_TIME, 'pt_BR.utf8')

'pt_BR.utf8'

Carregamento da base de dados e alteração do campo `Data` para um tipo de dado adequado para manipulação de datas. Na saída do comando `df.info()` é possível identificar que a coluna data agora possui o tipo de dado `datetime`.  

In [5]:
df = pd.read_csv('infodengue.csv') # Carrega a base de dados, o script para gerar este arquivo CSV se econtra no final deste notebook
df['Data'] = pd.to_datetime(df['Data']) # Converte a coluna data de string para tipo data do python
df['id'] = df['id'].astype(str)
df.info() # Imprime um resumo do Dataframe
df.head() # Imprime uma parte do Dataframe para ter uma ideia de como a informação está organizada

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46400 entries, 0 to 46399
Data columns (total 5 columns):
Data         46400 non-null datetime64[ns]
Casos        46400 non-null int64
id           46400 non-null object
Municipio    46400 non-null object
Estado       46400 non-null object
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.8+ MB


,Data,Casos,id,Municipio,Estado
0,2010-01-03,9,3300100,ANGRA DOS REIS,RJ
1,2010-01-10,5,3300100,ANGRA DOS REIS,RJ
2,2010-01-17,4,3300100,ANGRA DOS REIS,RJ
3,2010-01-24,11,3300100,ANGRA DOS REIS,RJ
4,2010-01-31,6,3300100,ANGRA DOS REIS,RJ


Iniciamos com uma visão geral da incidência dos casos de dengue em todo o estado do Rio de Janeiro. Isso nos dá uma idea de como ela está distribuida no tempo. É utilizado todo o período da base de dados obtida.

In [71]:
df_2 = df.groupby(['Data']).agg({'Casos':sum}) # Como num mesmo dia pode ter mais de um registro primeiro é necessário agrupar por data e somar as incidências de mesma data.
df_2.reset_index(inplace=True)

layout = go.Layout(title='Casos de dengue em todo o estado do Rio de Janeiro registrados<br> no período de ' +
                   df['Data'].min().strftime("%d/%m/%Y") + ' à ' + df['Data'].max().strftime("%d/%m/%Y") + '.')

traço = go.Scatter(x = df_2['Data'],
                   y = df_2['Casos'],
                   name = '',
                   showlegend = False)
dados = [traço]

figura = go.Figure(data=dados, layout=layout)
py.iplot(figura)

Partindo deste gráfico é possível identificar visualmente que ocorreram surtos ou maiores incidências de casos de dengue em todos os anos. Utilizando a biblioteca PeakUtils podemos evidenciar quando ocorreram os picos de surto ou maior incidência da doença em cada ano e o período em que ocorrem. O período em que ocorrem esses picos varia de estado para estado coincidindo com o período de chuvas da região em que o estado se encontra.

In [72]:
indices = peakutils.indexes(df_2['Casos'], thres=0.02, min_dist=30) 

df_picos = df_2.iloc[indices]

mes_min = pd.DatetimeIndex(df_picos['Data']).month.min()
mes_max = pd.DatetimeIndex(df_picos['Data']).month.max()

layout = go.Layout(title='<b>Ocorrência de surtos ou maior incidência dos casos de dengue.</b><br>De ' +
                   df['Data'].min().strftime("%d/%m/%Y") + ' à ' + df['Data'].max().strftime("%d/%m/%Y") + '.',
                   xaxis=dict(title='Os picos de incidência dos casos dengue ocorrem entre os meses de <b>' +
                              calendar.month_name[mes_min].capitalize() + ' e ' + calendar.month_name[mes_max].capitalize() + '.</b>'
                             )
                  )

traço = go.Scatter(x = df_2['Data'],
                   y = df_2['Casos'],
                   name = '',
                   showlegend = False)

picos = go.Scatter(x = df_2.iloc[indices]['Data'],
                   y = df_2.iloc[indices]['Casos'],
                   mode = 'markers + text',
                   marker = dict(size = 8,
                                 color = 'rgb(255,0,0)',
                                 symbol = 'cross'),
                   name = '',
                   textposition = 'top center',
                   textfont=dict(family='sans serif',
                                 size=14),
                   text = df_picos['Data'].dt.strftime("%-d %b %Y"),
                   showlegend = False)

dados = [traço, picos]

figura = go.Figure(data=dados, layout=layout)
py.iplot(figura)

Município que registrou o maior número de casos de dengue em um mesmo dia em toda a série história.

In [73]:
df.query('Casos==Casos.max()')

,Data,Casos,id,Municipio,Estado
31653,2012-04-22,4249,3304557,RIO DE JANEIRO,RJ


Municípios que registraram o maior número de casos de dengue nos meses de 2018

In [6]:
ano = 2018
df_3 = df[(df['Data'].dt.year == ano)]
df_3 = df_3.groupby(['Municipio', df['Data'].dt.month]).agg({'Casos':sum})
df_3.reset_index(inplace=True)
df_3.sort_values(['Data', 'Casos'], ascending=False, inplace=True)
df_3.rename(columns={'Data':'Mes'}, inplace=True)

df_3.drop_duplicates(subset=['Mes'], inplace=True)

df_3.sort_values('Mes', inplace=True)
df_3['Mes'] = df_3['Mes'].apply(lambda x: calendar.month_name[x].capitalize())

df_3

,Municipio,Mes,Casos
737,RIO DE JANEIRO,Janeiro,402
309,ITABORAÍ,Fevereiro,632
310,ITABORAÍ,Março,1201
311,ITABORAÍ,Abril,1380
741,RIO DE JANEIRO,Maio,1114
742,RIO DE JANEIRO,Junho,840
743,RIO DE JANEIRO,Julho,759
744,RIO DE JANEIRO,Agosto,367
745,RIO DE JANEIRO,Setembro,267
746,RIO DE JANEIRO,Outubro,232


Análise dos dez municípios com maior número de casos de dengue registrados em 2018.

In [75]:
ano = 2018
df_4 = df[(df['Data'].dt.year == ano)]
df_4 = df_4.groupby(['Municipio']).agg({'Casos':sum})
df_4.reset_index(inplace=True)
df_4.sort_values('Casos', ascending=False, inplace=True)
df_4 = df_4.head(10)
layout = go.Layout(title = 'Dez municípios com maior ocorrência de casos de dengue em ' + str(ano) + '.' )
dados = [go.Bar(x = df_4['Municipio'], y = df_4['Casos'], orientation = 'v')]
figura = go.Figure(data = dados, layout = layout)
py.iplot(figura)

Disposiçao geográfica dos casos de denguem no Rio de Jnaiero registrados em 2018. A escala foi limitada a mil apenas para efeito de visuailação já que a maior parte dos municípios está abaixo deste valor o que fica evidente no mapa apresentado. Para esta análise foram utilizados a biblioteca Folium e um arquivo [GeoJson](https://github.com/tbrugz/geodata-br/blob/master/geojson/geojs-33-mun.json) contendo os limites geográficos de cada município do estado do Rio de Janeiro, este arquivo está disponível no repositório Github [Geodata BR - Brasil](https://github.com/tbrugz/geodata-br).

In [9]:
ano = 2018
df_5 = df[(df['Data'].dt.year == ano)]
df_5 = df_5.groupby(['id', 'Municipio']).agg({'Casos':sum})
df_5.reset_index(inplace=True)
df_5.sort_values('Casos', ascending=False, inplace=True)

geojson = 'geojs-33-mun.json'
df_geojson = pd.read_json(geojson)
df_5_dict = df_5.set_index('id')['Casos'].to_dict()

# Escala de cores utilizada na representação do mapa
color_scale = LinearColormap(['#FFFFAA', '#FFFF00', '#FFE200', '#FFC600', '#FFAA00', '#FF8D00', '#FF7100', '#FF5500', '#FF3800', '#FF1C00', '#FF0000'],
                             vmin = 0, vmax = 1000, caption='Número de casos de dengue em ' + str(ano) + ' de zero a mil ou mais casos.')

# Função para colorir cada município de acordo com o número de casos de dengue,
# para municípios com zero casos de dengue atribui a cor branca e para munnicípios
# que não possuam informações na base dados atribui a cor cinza.
def style_function(feature):
    value = df_5_dict.get(feature['properties']['id'])
    return {
        'fillColor': '#7c7c7c' if value is None else ('#FFFFFF' if value == 0 else color_scale(value)),
        'fillOpacity': 0.8,
        'color': 'black',
        'weight': 0.5,
    }    

# Função para permitir selecionar um município clicando sobre ele alterando sua cor ao passar com o mouse sobre ele
def highlight_function(feature):
    return {
        'fillColor': 'blue',
        'fillOpacity': 0.4,
        'color': 'blue',
        'weight': 1.5,
    }

# Cria um mapa utiliando o Folium centralizado no estado do Rio de Janeiro e com zoom fixo
m = folium.Map(location=[-22,-43], zoom_start=8, max_zoom=8, min_zoom=8)

# Função para adicionar ao mapa o poligno que representa os limites geográficos de cada município. Esta função
# também adiciona um popup para cada município com informações sobre ele.
for index, row in df_geojson.iterrows():
    cod = row['features']['properties']['id']
    casos = df_5_dict.get(cod)
    municipio = row['features']['properties']['name']
    gj = folium.GeoJson(
        row['features'],
        name=(municipio),
        overlay=True,
        style_function=style_function,
        highlight_function=highlight_function
    )
    if casos is None:
        # Popup para municípios que não possuem informações na basse de dados
        folium.Popup('O município <b>{}</b> não possui dados sobre a dengue. Mais informações em <a href="https://info.dengue.mat.br/participe/" target="_blank"> Info Dengue </a>.'.format(municipio)).add_to(gj)
    else:
        # Popup para informar o nome do município e o número da casos de engue
        folium.Popup('Município: <b>{}</b><br>Casos: <b>{}</b>'.format(municipio, casos)).add_to(gj)
    gj.add_to(m)
    
# Exibe a escala de cores no mapa no canto direito superior  
m.add_child(color_scale)

# Exibe o mapa na tela
m

<a id='script'></a> Shell script utilizado para gerar a base de dados CSV contendo a informação de todos os municípios de um mesmo estado em um só arquivo. Dependendo do estado selecionado por levar alguns minutos para baixar as informações de todos os municípios.

In [69]:
%%bash
#!/bin/bash
# Script utilizado para unificar as bases de todos os municípios já que a fonte utilizada não disponibiliza a informação centralizada desta forma;
IFS=$'\n'

# Configura os dados da informação a ser baixada
ESTADO='RJ'
FORMATO='csv'
DOENCA='dengue'
SEMANA_INICIAL='1'
SEMANA_FINAL='50'
ANO_INICIAL='0' # 0 baixa a data mais antiga que estiver disponível
ANO_FINAL='2018'

# Cria um novo arquivo para armazenar a informação a ser analisada
echo "Data,Casos,id,Municipio,Estado" > infodengue.$FORMATO

# Obtém uma lista dos códigos IBGE dos municipios necessário para obter o arquivo CSV de cada municipio
COD_MUNICIPIOS=$(wget --quiet -O - https://info.dengue.mat.br/services/api | grep -E "[0-9]{7} - [A-Z ]* - $ESTADO" | sed -e 's/^[ ]*//' -e 's/ - /-/g')

# Percorre toda a lista de códigos dos municipios do estado selecionado
for i in $COD_MUNICIPIOS
do
    echo $i
    # Baixa o arquivo CSV do municipio utilizando seu código IBGE
    wget --quiet -O tmp "https://info.dengue.mat.br/api/alertcity?geocode=$(echo $i | cut -d'-' -f 1)&disease=$DOENCA&format=$FORMATO&ew_start=$SEMANA_INICIAL&ew_end=$SEMANA_FINAL&ey_start=$ANO_INICIAL&ey_end=$ANO_FINAL"
    
    # Foi verificado que existem dois tipos de cabeçalho nas bases analisadas.
    if [[ "$(head -1 tmp)" == *"data_iniSE"* ]]
    then
        # Para cabeçalhos onde o campo data está nomeado como "data_iniSE"
        cut -d',' -f 1,6 tmp > tmp2
    else
        # Para os demais casos onde o campo data está denominado apenas como "data"
        cut -d, -f 4,6 tmp > tmp2
    fi
    
    # Concatena a informação obtida de cada municipio inserindo as colunas Municipio e Estado ao qual esta informação pertence.
    tail -n +2 tmp2 | sed "s/$/,\"$(echo $i | cut -d'-' -f 1)\",$(echo $i | cut -d'-' -f 2),$(echo $i | cut -d'-' -f 3)/g" >> infodengue.$FORMATO
done

# Remove os arquivos temporários
rm tmp*


3300100-ANGRA DOS REIS-RJ
3300159-APERIBÉ-RJ
3300209-ARARUAMA-RJ
3300225-AREAL-RJ
3300233-ARMAÇÃO DOS BÚZIOS-RJ
3300258-ARRAIAL DO CABO-RJ
3300308-BARRA DO PIRAÍ-RJ
3300407-BARRA MANSA-RJ
3300456-BELFORD ROXO-RJ
3300506-BOM JARDIM-RJ
3300605-BOM JESUS DO ITABAPOANA-RJ
3300704-CABO FRIO-RJ
3300803-CACHOEIRAS DE MACACU-RJ
3300902-CAMBUCI-RJ
3301009-CAMPOS DOS GOYTACAZES-RJ
3301108-CANTAGALO-RJ
3300936-CARAPEBUS-RJ
3301157-CARDOSO MOREIRA-RJ
3301207-CARMO-RJ
3301306-CASIMIRO DE ABREU-RJ
3300951-COMENDADOR LEVY GASPARIAN-RJ
3301405-CONCEIÇÃO DE MACABU-RJ
3301504-CORDEIRO-RJ
3301603-DUAS BARRAS-RJ
3301702-DUQUE DE CAXIAS-RJ
3301801-ENGENHEIRO PAULO DE FRONTIN-RJ
3301850-GUAPIMIRIM-RJ
3301876-IGUABA GRANDE-RJ
3301900-ITABORAÍ-RJ
3302007-ITAGUAÍ-RJ
3302056-ITALVA-RJ
3302106-ITAOCARA-RJ
3302205-ITAPERUNA-RJ
3302254-ITATIAIA-RJ
3302270-JAPERI-RJ
3302304-LAJE DO MURIAÉ-RJ
3302403-MACAÉ-RJ
3302452-MACUCO-RJ
3302502-MAGÉ-RJ
3302601-MANGARATIBA-RJ
3302700-MARICÁ-RJ
3302809-MENDES-RJ
3302858-MESQUIT

### Referências
---
[Análise rápida dos dados de óbito por Dengue em todo o Brasil](https://github.com/dataAt/introducao-analise-de-dados/blob/master/src/Python/analise_dengue/obitos-dengue.ipynb)  
[Info Dengue API](https://info.dengue.mat.br/services/api/doc)  
[A Basic Pandas Dataframe Tutorial for Beginners](https://www.marsja.se/pandas-dataframe-read-csv-excel-subset/)  
[Rio anuncia epidemia de dengue](http://g1.globo.com/rio-de-janeiro/noticia/2012/04/rio-anuncia-epidemia-de-dengue.html)  
[Número de mortes por dengue no RJ em 2013 sobe para 28](http://g1.globo.com/rio-de-janeiro/noticia/2013/05/numero-de-mortes-por-dengue-no-rj-em-2013-sobe-para-28.html)  
[Pandas filter dataframe rows with a specific year](https://stackoverflow.com/questions/46878156/pandas-filter-dataframe-rows-with-a-specific-year)  
[Pandas group-by and sum](https://stackoverflow.com/questions/39922986/pandas-group-by-and-sum)  
[Sort Pandas Dataframe by Date](https://stackoverflow.com/questions/28161356/sort-pandas-dataframe-by-date)  
[Time Series in Pandas](https://plot.ly/pandas/time-series/)  
[Plot with seaborn after groupby command in pandas](https://www.kaggle.com/questions-and-answers/55356)  
[Peak Finding in Python](https://plot.ly/python/peak-finding/)  
[Horizontal Bar Charts in Python](https://plot.ly/python/horizontal-bar-charts/)  
[Choropleth Maps in Python](https://plot.ly/python/choropleth-maps/#united-states-choropleth-map)  
[Advanced Jupyter Notebook Tricks — Part I](https://blog.dominodatalab.com/lesser-known-ways-of-using-notebooks/)
[Setting the Title, Legend Entries, and Axis Titles in Python](https://plot.ly/python/figure-labels/)  
[Plotly legend title](https://stackoverflow.com/questions/45555266/plotly-legend-title)  
[Plotly User Guide in Python](https://plot.ly/python/user-guide/)  
[How to reference a IPython notebook cell in markdown?](https://stackoverflow.com/questions/28080066/how-to-reference-a-ipython-notebook-cell-in-markdown)  
[Offline Plots in Plotly in Python](https://plot.ly/python/offline/)  
[PeakUtils tutorial](https://peakutils.readthedocs.io/en/latest/tutorial_a.html)  
[Select Rows & Columns by Name or Index in DataFrame using loc & iloc | Python Pandas](https://thispointer.com/select-rows-columns-by-name-or-index-in-dataframe-using-loc-iloc-python-pandas/)  
[Python's strftime directives](http://strftime.org)  
[How to plot all peaks using Python](https://stackoverflow.com/questions/44502658/how-to-plot-all-peaks-using-python)  
[Peak Finding in Python](https://plot.ly/python/peak-finding/)  
[PeakUtils — A peak finding algorithm](https://astroumd.github.io/admit/module/admit.util.peakfinder/PeakUtils.html)  
[Lidando com datas e horários no Python](http://blog.alura.com.br/lidando-com-datas-e-horarios-no-python/)  
[Formatação de data na língua local no Python](https://ricobl.wordpress.com/2010/10/24/formatacao-de-data-na-lingua-local-no-python/)  
[Hover Text and Formatting in Python](https://plot.ly/python/hover-text-and-formatting/)  
[Text and Annotations in plotly.js](https://plot.ly/javascript/text-and-annotations/)  
[Text and Annotations in Python](https://plot.ly/python/text-and-annotations/)  
[Extract month and year from column in Pandas, create new column](https://erikrood.com/Python_References/extract_month_year_pandas_final.html)  
[Get month name from number](https://stackoverflow.com/questions/6557553/get-month-name-from-number)  
[Dealing with datetimes like a pro in Pandas](https://medium.com/jbennetcodes/dealing-with-datetimes-like-a-pro-in-pandas-b80d3d808a7f)  
[plotly.offline.iplot gives a large blank field as its output - why?](https://stackoverflow.com/questions/48560138/plotly-offline-iplot-gives-a-large-blank-field-as-its-output-why)  
[Links, Text, and HTML in Graphs](https://help.plot.ly/adding-HTML-and-links-to-charts/)  
[Arquivos Geojson com perímetros dos municípios brasileiros por estado ( Brasil / Brazil )](https://github.com/tbrugz/geodata-br)  
[Map this… Comparing choropleth maps in Folium and Tableau Public](https://medium.com/dataexplorations/map-this-comparing-choropleth-maps-in-folium-and-tableau-public-9c15f09f5e79)  
[GeoJSON and choropleth](https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/GeoJSON_and_choropleth.ipynb#Using-choropleth-method)  
[Set color for missing values in folium choropleth](https://stackoverflow.com/questions/51961848/set-color-for-missing-values-in-folium-choropleth)  
[GeoJSON popup not working when GeoJSON in FeatureGroup #904](https://github.com/python-visualization/folium/issues/904)  
[How to create Popups](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Popups.ipynb)  
[Folium](http://python-visualization.github.io/folium/docs-v0.5.0/modules.html)  
[Python - Folium Choropleth Map - colors incorrect](https://stackoverflow.com/questions/52911688/python-folium-choropleth-map-colors-incorrect)  
[Pandas: get the first occurrence grouping by keys](https://stackoverflow.com/questions/45057967/pandas-get-the-first-occurrence-grouping-by-keys)  
[Drop all duplicate rows in Python Pandas](https://stackoverflow.com/questions/23667369/drop-all-duplicate-rows-in-python-pandas)  
[string capitalize() in Python](https://www.geeksforgeeks.org/string-capitalize-python/)  
[python/pandas: convert month int to month name](https://stackoverflow.com/questions/37625334/python-pandas-convert-month-int-to-month-name)  
[Time Series in Python ](https://plot.ly/python/time-series/)  
[How to display plotly outputs on Github?](https://github.com/plotly/plotly.py/issues/931)  
[I updated my ipython notebook gist, but why didn't nbviewer update?](https://stackoverflow.com/questions/29034001/i-updated-my-ipython-notebook-gist-but-why-didnt-nbviewer-update)